In [1]:
%%bash
mkdir -p data
pushd data
if [ -d "20news-bydate-train" ]
then
  echo "The data has already been downloaded..."
else
  wget http://qwone.com/%7Ejason/20Newsgroups/20news-bydate.tar.gz
  tar xfv 20news-bydate.tar.gz
  rm 20news-bydate.tar.gz
fi
echo "Lets take a look at the groups..."
ls 20news-bydate-train/
popd

~/Dropbox/TopicModeling/Code/Python/data ~/Dropbox/TopicModeling/Code/Python
20news-bydate-test/
20news-bydate-test/alt.atheism/
20news-bydate-test/alt.atheism/53265
20news-bydate-test/alt.atheism/53339
20news-bydate-test/alt.atheism/53260
20news-bydate-test/alt.atheism/53340
20news-bydate-test/alt.atheism/53333
20news-bydate-test/alt.atheism/53302
20news-bydate-test/alt.atheism/53313
20news-bydate-test/alt.atheism/53293
20news-bydate-test/alt.atheism/53297
20news-bydate-test/alt.atheism/53315
20news-bydate-test/alt.atheism/53320
20news-bydate-test/alt.atheism/53324
20news-bydate-test/alt.atheism/53328
20news-bydate-test/alt.atheism/53325
20news-bydate-test/alt.atheism/53322
20news-bydate-test/alt.atheism/53326
20news-bydate-test/alt.atheism/53261
20news-bydate-test/alt.atheism/53327
20news-bydate-test/alt.atheism/53329
20news-bydate-test/alt.atheism/53321
20news-bydate-test/alt.atheism/53068
20news-bydate-test/alt.atheism/53338
20news-bydate-test/alt.atheism/53257
20news-bydate-test/a

--2019-07-01 11:48:59--  http://qwone.com/%7Ejason/20Newsgroups/20news-bydate.tar.gz
Resolving qwone.com (qwone.com)... 173.48.216.20
Connecting to qwone.com (qwone.com)|173.48.216.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14464277 (14M) [application/x-gzip]
Saving to: ‘20news-bydate.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 78.5K 2m59s
    50K .......... .......... .......... .......... ..........  0% 5.45M 91s
   100K .......... .......... .......... .......... ..........  1% 1.28M 64s
   150K .......... .......... .......... .......... ..........  1% 5.32M 48s
   200K .......... .......... .......... .......... ..........  1% 5.04M 39s
   250K .......... .......... .......... .......... ..........  2% 7.74M 33s
   300K .......... .......... .......... .......... ..........  2% 9.76M 28s
   350K .......... .......... .......... .......... ..........  2% 10.1M 25s
   400K .......... .......... .......... .......... ..


**Exploring the dataset**

Each group dir has a set of files:


In [2]:
ls -lah data/20news-bydate-train/sci.space | tail  -n 5

-rw-r--r--  1 santhilata santhilata 1.5K Mar 18  2003 61250
-rw-r--r--  1 santhilata santhilata  889 Mar 18  2003 61252
-rw-r--r--  1 santhilata santhilata 1.2K Mar 18  2003 61264
-rw-r--r--  1 santhilata santhilata 1.7K Mar 18  2003 61308
-rw-r--r--  1 santhilata santhilata 1.4K Mar 18  2003 61422


Lets take a peak at one email:

In [3]:
!head data/20news-bydate-train/sci.space/61422 -n 20

From: ralph.buttigieg@f635.n713.z3.fido.zeta.org.au (Ralph Buttigieg)
Subject: Why not give $1 billion to first year-lo
Organization: Fidonet. Gate admin is fido@socs.uts.edu.au
Lines: 34

Original to: keithley@apple.com
G'day keithley@apple.com

21 Apr 93 22:25, keithley@apple.com wrote to All:

 kc> keithley@apple.com (Craig Keithley), via Kralizec 3:713/602


 kc> But back to the contest goals, there was a recent article in AW&ST
about a
 kc> low cost (it's all relative...) manned return to the moon.  A General
 kc> Dynamics scheme involving a Titan IV & Shuttle to lift a Centaur upper
 kc> stage, LEV, and crew capsule.  The mission consists of delivering two
 kc> unmanned payloads to the lunar surface, followed by a manned mission.
 kc> Total cost:  US was $10-$13 billion.  Joint ESA(?)/NASA project was


**Loading and Tokenizing the corpus**

In [4]:
from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd

path_to_folder = 

In [5]:
# quick and dirty....
EMAIL_REGEX = re.compile(r"[a-z0-9\.\+_-]+@[a-z0-9\._-]+\.[a-z]*")
FILTER_REGEX = re.compile(r"[^a-z '#]")
TOKEN_MAPPINGS = [(EMAIL_REGEX, "#email"), (FILTER_REGEX, ' ')]

def tokenize_line(line):
    res = line.lower()
    for regexp, replacement in TOKEN_MAPPINGS:
        res = regexp.sub(replacement, res)
    return res.split()
    
def tokenize(lines, token_size_filter=2):
    tokens = fp.mapcat(tokenize_line, lines)
    return [t for t in tokens if len(t) > token_size_filter]
    

def load_doc(filename):
    group, doc_id = filename.split('/')[-2:]
    with open(filename, errors='ignore') as f:
        doc = f.readlines()
    return {'group': group,
            'doc': doc,
            'tokens': tokenize(doc),
            'id': doc_id}


docs = pd.DataFrame(list(map(load_doc, glob('data/20news-bydate-train/*/*')))).set_index(['group','id'])
docs.head()

doc  \
group                 id                                                         
comp.sys.mac.hardware 51656  [From: erme@pobox.upenn.edu (Erme Maula)\n, Su...   
                      51797  [From: betts@netcom.com (Jonathan Betts)\n, Su...   
                      51729  [From: cherkaue@ee.rochester.edu (Brian Cherka...   
                      51716  [From: pilon@aix02.ecs.rpi.edu (T.J. Pilon)\n,...   
                      51588  [From: wlyle@sju.edu (Wayne Lyle)\n, Subject: ...   

                                                                        tokens  
group                 id                                                        
comp.sys.mac.hardware 51656  [from, #email, erme, maula, subject, macs, suc...  
                      51797  [from, #email, jonathan, betts, subject, ghost...  
                      51729  [from, #email, brian, cherkauer, subject, iivx...  
                      51716  [from, #email, pilon, subject, iicx, won't, tu...  
                      51588  [from, #email, wayne, lyle, subject, driver, f...

**Creating the dictionary, and bag of words corpus**

In [6]:
def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=5, no_above=0.5):
    
    print('Building dictionary...')
    dictionary = Dictionary(docs)
    stopwords = nltk_stopwords().union(additional_stopwords)
    stopword_ids = map(dictionary.token2id.get, stopwords)
    dictionary.filter_tokens(stopword_ids)
    dictionary.compactify()
    dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
    dictionary.compactify()

    print('Building corpus...')
    corpus = [dictionary.doc2bow(doc) for doc in docs]

    return dictionary, corpus

In [7]:
dictionary, corpus = prep_corpus(docs['tokens'])

Building dictionary...
Building corpus...


In [8]:
MmCorpus.serialize('newsgroups.mm', corpus)
dictionary.save('newsgroups.dict')

/home/santhilata/anaconda3/envs/r-tutorial/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


**Fitting the LDA model**

In [9]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=50, passes=10)
                                      
lda.save('newsgroups_50_lda.model')

CPU times: user 14min 45s, sys: 15min 7s, total: 29min 53s
Wall time: 4min 36s


In [16]:
from pprint import pprint
pprint(lda.print_topics())

[(16,
  '0.038*"illinois" + 0.031*"animation" + 0.028*"gordon" + 0.026*"university" '
  '+ 0.025*"article" + 0.025*"banks" + 0.024*"computer" + 0.021*"urbana" + '
  '0.021*"marc" + 0.021*"science"'),
 (44,
  '0.033*"sin" + 0.027*"polygon" + 0.014*"marriage" + 0.014*"den" + '
  '0.012*"doctrine" + 0.012*"weiss" + 0.010*"kendig" + 0.010*"labor" + '
  '0.010*"routine" + 0.010*"article"'),
 (40,
  '0.017*"like" + 0.016*"would" + 0.014*"good" + 0.013*"get" + 0.013*"i\'m" + '
  '0.011*"car" + 0.011*"one" + 0.009*"i\'ve" + 0.009*"know" + 0.009*"want"'),
 (15,
  '0.067*"space" + 0.064*"drive" + 0.035*"scsi" + 0.026*"disk" + 0.024*"nasa" '
  '+ 0.023*"drives" + 0.022*"hard" + 0.019*"earth" + 0.016*"data" + '
  '0.013*"floppy"'),
 (2,
  '0.017*"men" + 0.015*"sex" + 0.014*"cramer" + 0.012*"sexual" + '
  '0.011*"homosexual" + 0.009*"group" + 0.009*"homosexuality" + '
  '0.008*"homosexuals" + 0.008*"groups" + 0.008*"article"'),
 (39,
  '0.027*"oil" + 0.016*"energy" + 0.011*"food" + 0.011*"nuclear" 


**Visualizing the model with pyLDAvis**

Okay, the moment we have all been waiting for is finally here! You'll notice in the visualization that we have a few junk topics that would probably disappear after better preprocessing of the corpus. 

In [10]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [11]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/home/santhilata/anaconda3/envs/r-tutorial/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))



**Fitting the HDP model**

We can both visualize LDA models as well as gensim HDP models with pyLDAvis.

The difference between HDP and LDA is that HDP is a non-parametric method. Which means that we don't need to specify the number of topics. HDP will fit as many topics as it can and find the optimal number of topics by itself.


In [12]:
%%time
# The optional parameter T here indicates that HDP should find no more than 50 topics
# if there exists any.
hdp = models.hdpmodel.HdpModel(corpus, dictionary, T=50)
                                      
hdp.save('newsgroups_hdp.model')



/home/santhilata/anaconda3/envs/r-tutorial/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 1min 32s, sys: 1min 5s, total: 2min 37s
Wall time: 45.4 s



**Visualizing the HDP model with pyLDAvis**

As for the LDA model, in order to prepare the visualization you only need to pass it your model, the corpus, and the associated dictionary.


In [13]:
vis_data = gensimvis.prepare(hdp, corpus, dictionary)
pyLDAvis.display(vis_data)

/home/santhilata/anaconda3/envs/r-tutorial/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
